In [1]:
import pandas as pd
import numpy as np
import datetime
import os
# Confirmed with YG, DCart campaigns after Jan 1 only, 2020-02-03 ~ 1 p.m.
print(datetime.datetime.now())
os.getcwd()

2020-02-03 14:47:13.491176


'/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Planner_Request/BR_audience_furniture_browser'

In [2]:
# Download through the shared GoogleSheet, 2020-02-03 around 11 a.m.
df_campaing_dates=pd.read_excel("/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Planner_Request/BR_audience_furniture_browser/by_overlap_with_dcart/D Cart Weekly Budget 1121.xlsx",
                               dtype=str,sheet_name="Sheet2",skiprows=1)
df_campaing_dates=df_campaing_dates.iloc[:,:5]
df_campaing_dates['Unnamed: 0']=df_campaing_dates['Unnamed: 0'].fillna(method='ffill')
df_campaing_dates['Unnamed: 1']=df_campaing_dates['Unnamed: 1'].fillna(method='ffill')
df_campaing_dates['Unnamed: 0']=df_campaing_dates['Unnamed: 0'].str.capitalize()
df_campaing_dates['Unnamed: 1']=df_campaing_dates['Unnamed: 1'].str.capitalize()

df_campaing_dates['week']=df_campaing_dates['Unnamed: 1']+"_"+df_campaing_dates['Unnamed: 0']

df_campaing_dates['StartDate']=df_campaing_dates['StartDate'].fillna("nan")
df_campaing_dates['EndDate']=df_campaing_dates['EndDate'].fillna("nan")

df_campaing_dates['StartDate']=df_campaing_dates['StartDate'].apply(lambda x: x[:10])
df_campaing_dates['EndDate']=df_campaing_dates['EndDate'].apply(lambda x: x[:10])
df_campaing_dates.head(2)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,StartDate,EndDate,week
0,W1,Weekly,Account1,2019-11-14,2019-11-23,Weekly_W1
1,W1,Nonweekly,Account2,2019-11-18,2019-11-23,Nonweekly_W1


In [3]:
# df_2_weeks_campaign=df_campaing_dates[df_campaing_dates['EndDate']>="2020-01-01"]
df_2_weeks_campaign=df_campaing_dates[pd.notnull(df_campaing_dates['StartDate'])]
df_2_weeks_campaign=df_2_weeks_campaign[df_2_weeks_campaign['StartDate']>="2020-01-01"]
df_2_weeks_campaign=df_2_weeks_campaign[df_2_weeks_campaign['StartDate']<="2020-01-20"]

# Start after 200101

print(df_2_weeks_campaign.shape)
list_needed_weeks=df_2_weeks_campaign['week'].tolist()
len(set(list_needed_weeks))

(13, 6)


13

In [4]:
df_overlapped_ids=pd.read_csv("./by_overlap_with_dcart/overlapwithdcart.csv",dtype=str)
df_overlapped_ids['campaign_list']=df_overlapped_ids['week'].apply(lambda x: x.split(","))
print(df_overlapped_ids.shape)
print(df_overlapped_ids['customer_id_hashed'].nunique())

df_overlapped_ids.head(2)

(179247, 3)
179247


,customer_id_hashed,week,campaign_list
0,00000435bfb3bf42e3beb4c9b3942c552d09f0e49e5a75...,"Weekly_W4, Weekly_W6, Nonshopper_W1, Nonshoppe...","[Weekly_W4, Weekly_W6, Nonshopper_W1, Nonsh..."
1,0001177d3a03ae7f2c1ed77ef69820b92321baa6981245...,Weekly_W8,[Weekly_W8]


In [5]:
df_test=pd.read_csv("./BL_furniture_browser_test_group_JL_+str(datetime.datetime.now().date()).csv",dtype=str)
df_control=pd.read_csv("./BL_furniture_browser_control_group_JL_+str(datetime.datetime.now().date()).csv",dtype=str)

print(df_test.shape,df_control.shape)

df_test.head(2)

(489861, 5) (54430, 5)


,index,customersummary_mastercustomerid,Email,customer_id_hashed,execution_id
0,0,KFK_0_8017815,ad5f11c041c89839c09204f9ce34f5d97ea8ddaa81f1f0...,83228bfe2f6d62bb0218751ceb63aa8551e9fd89a999f1...,1578944138509
1,2,KFK_0_37564566,c612188e441b8fef2e5eed0c918fec1d12e23f801b9e69...,94245ca46539f480c2490a5e3373f237dbc455094f85b7...,1578944138509


In [6]:
df_group_1=df_test[['customer_id_hashed']].drop_duplicates()
df_group_1['group']='test'
df_group_2=df_control[['customer_id_hashed']].drop_duplicates()
df_group_2['group']='control'

df_group=df_group_1.append(df_group_2)
print(df_group.shape,df_group['customer_id_hashed'].nunique())

df_total_id=df_group.groupby(['group'])['customer_id_hashed'].count().to_frame().reset_index().rename(columns={"customer_id_hashed":"total_ids"})

(544291, 2) 544291


In [7]:
df_group=pd.merge(df_group,df_overlapped_ids,on="customer_id_hashed",how="left")
df_group['DCart_Type_all']=np.where(pd.isnull(df_group['week']),"NotDCart_but_AgilOne","OverlapWithDCart")
print(df_group.shape)

(544291, 5)


In [8]:
df_in_DCart=df_group[df_group['DCart_Type_all']=="OverlapWithDCart"]
dict_id_DCart_selected_campaign_weeks=df_in_DCart.set_index("customer_id_hashed").to_dict()["week"]
counter_InSelectedCampaign=0
counter_OtherDCartCampaign=0

for hashed_id_key in dict_id_DCart_selected_campaign_weeks.keys():
    for week_name in list_needed_weeks:
        if week_name in dict_id_DCart_selected_campaign_weeks[hashed_id_key]:
            dict_id_DCart_selected_campaign_weeks.update({hashed_id_key:"In_DCart2020"})
            counter_InSelectedCampaign+=1
    if dict_id_DCart_selected_campaign_weeks[hashed_id_key]!="In_DCart2020":
        dict_id_DCart_selected_campaign_weeks.update({hashed_id_key:"Not_DCart2020"})
        counter_OtherDCartCampaign+=1

set(dict_id_DCart_selected_campaign_weeks.values())
print(len(dict_id_DCart_selected_campaign_weeks))
print("counter_InSelectedCampaign",counter_InSelectedCampaign)
print("counter_OtherDCartCampaign",counter_OtherDCartCampaign)

150098
counter_InSelectedCampaign 23598
counter_OtherDCartCampaign 126500


In [9]:
df_group['DCart_Type_2020']=df_group['customer_id_hashed'].apply(lambda x: dict_id_DCart_selected_campaign_weeks[x] if x in dict_id_DCart_selected_campaign_weeks.keys() else "NotDCart_but_AgilOne")

'''
df_group_merge=df_group[['customer_id_hashed','group','DCart_Type_all','DCart_Type_2020']]

df_group_merge.shape
'''

"\ndf_group_merge=df_group[['customer_id_hashed','group','DCart_Type_all','DCart_Type_2020']]\n\ndf_group_merge.shape\n"

In [10]:
df_total_id_overall=df_group.groupby(["group","DCart_Type_all"])['customer_id_hashed'].count().to_frame().reset_index()
print(df_total_id_overall['customer_id_hashed'].sum())
df_total_id_overall

544291


,group,DCart_Type_all,customer_id_hashed
0,control,NotDCart_but_AgilOne,39454
1,control,OverlapWithDCart,14976
2,test,NotDCart_but_AgilOne,354739
3,test,OverlapWithDCart,135122


In [11]:
df_total_id_2020=df_group.groupby(["group","DCart_Type_2020"])['customer_id_hashed'].count().to_frame().reset_index()
print(df_total_id_2020['customer_id_hashed'].sum())
df_total_id_2020

544291


,group,DCart_Type_2020,customer_id_hashed
0,control,In_DCart2020,2312
1,control,NotDCart_but_AgilOne,39454
2,control,Not_DCart2020,12664
3,test,In_DCart2020,21286
4,test,NotDCart_but_AgilOne,354739
5,test,Not_DCart2020,113836


# 1. Overall

In [12]:
# QC purpose only

df_recevied_AgilOne=pd.read_csv("/home/jian/BigLots/AgilOne/Division_9_Browse_BigLots_20200113073732.csv",dtype=str)
df_recevied_AgilOne=df_recevied_AgilOne.rename(columns={"customersummary_c_primaryscnhash":"customer_id_hashed"})
df_recevied_AgilOne=pd.merge(df_recevied_AgilOne,df_overlapped_ids,on="customer_id_hashed",how="left")
print(df_recevied_AgilOne[pd.notnull(df_recevied_AgilOne['week'])].shape)
df_recevied_AgilOne.head(2)

(179247, 6)


,customersummary_mastercustomerid,customersummary_emailhash,customer_id_hashed,execution_id,week,campaign_list
0,KFK_0_8017815,ad5f11c041c89839c09204f9ce34f5d97ea8ddaa81f1f0...,83228bfe2f6d62bb0218751ceb63aa8551e9fd89a999f1...,1578944138509,"Nonshopper_W1, Nonshopper_W2, Nonshopper_W4, N...","[Nonshopper_W1, Nonshopper_W2, Nonshopper_W4..."
1,KFK_0_39127329,f3bc025a0bb51d1b097ea5a956357114a7948145fdd506...,68d99d05642aef443fe7f99c278f971619dd50fbcfa0c3...,1578944138509,NaN,NaN


In [13]:
df_sales_by_day_1=pd.read_table("/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-01-18/MediaStormDailySales20200121-111749-649.txt",
                                sep="|",dtype=str)
df_sales_by_day_2=pd.read_table("/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-01-25/MediaStormDailySales20200128-111758-074.txt",
                                sep="|",dtype=str)
df_sales_by_day=df_sales_by_day_1.append(df_sales_by_day_2)

del df_sales_by_day_1
del df_sales_by_day_2

df_group=df_group[['customer_id_hashed','group','DCart_Type_all','DCart_Type_2020']]

In [14]:
df_sales_by_day['item_transaction_amt']=df_sales_by_day['item_transaction_amt'].astype(float)
df_sales_by_day=df_sales_by_day[pd.notnull(df_sales_by_day['customer_id_hashed'])]
df_sales_by_day=pd.merge(df_sales_by_day,df_group,on="customer_id_hashed",how="left")
df_sales_by_day['group']=df_sales_by_day['group'].fillna('others')
df_sales_by_day['DCart_Type_all']=df_sales_by_day['DCart_Type_all'].fillna('others')
df_sales_by_day['DCart_Type_2020']=df_sales_by_day['DCart_Type_2020'].fillna('others')



In [15]:
df_taxonomy_div=pd.read_table("/home/jian/BigLots/static_files/ProductTaxonomy/MediaStormProductTaxonomy20200101-135600-916.txt",dtype=str,sep="|")
df_taxonomy_div=df_taxonomy_div[['division_id','class_code_id','subclass_id']].drop_duplicates()

df_sales_by_day=pd.merge(df_sales_by_day,df_taxonomy_div,on=['class_code_id','subclass_id'],how="left")
# df_sales_by_day['store_type']=np.where(df_sales_by_day['location_id']=="6990","online","instore")

In [16]:
df_sales_by_day['DCart_Type_all'].unique()

array(['others', 'NotDCart_but_AgilOne', 'OverlapWithDCart'], dtype=object)

In [17]:
df_sales_by_day['DCart_Type_2020'].unique()

array(['others', 'NotDCart_but_AgilOne', 'Not_DCart2020', 'In_DCart2020'],
      dtype=object)

In [18]:
df_sales_by_day.head(2)

,location_id,transaction_dt,transaction_id,customer_id_hashed,class_code_id,subclass_id,item_id,item_transaction_units,item_transaction_amt,group,DCart_Type_all,DCart_Type_2020,division_id
0,1027,2020-01-18,2108,51cc9db6c31bf222baea4e50b9a6b1be2c7c4ec833bcfe...,14001,1,140008972,1,1.6,others,others,others,2
1,5269,2020-01-18,7147,3fb7f19d961a20055d0fb89d584bd36b35f26c56e29e32...,12007,4,810358789,1,2.8,others,others,others,2


# By_Day, Overall DCartOverlap

In [19]:
agg_fun={"customer_id_hashed":"nunique","item_transaction_amt":"sum"}
df_total_sales=df_sales_by_day.groupby(['transaction_dt','group','DCart_Type_all'])['customer_id_hashed','item_transaction_amt'].agg(agg_fun).reset_index()
df_total_trans=df_sales_by_day[['group','DCart_Type_all','location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
df_total_trans['trans']=1
df_total_trans=df_total_trans.groupby(['transaction_dt','DCart_Type_all','group'])['trans'].sum().to_frame().reset_index()
df_total=pd.merge(df_total_sales,df_total_trans,on=['transaction_dt','DCart_Type_all','group'],how="outer")
df_total=df_total.rename(columns={"customer_id_hashed":"total_shoppers","item_transaction_amt":"total_sales","trans":"total_trans"})
df_total['total_AOV']=df_total['total_sales']/df_total['total_trans']

df_division_sales=df_sales_by_day.groupby(['transaction_dt','group','DCart_Type_all','division_id'])['customer_id_hashed','item_transaction_amt'].agg(agg_fun).reset_index()
df_division_trans=df_sales_by_day[['group','DCart_Type_all','location_id','transaction_dt','transaction_id','customer_id_hashed','division_id']].drop_duplicates()
df_division_trans['trans']=1
df_division_trans=df_division_trans.groupby(['transaction_dt','DCart_Type_all','group','division_id'])['trans'].sum().to_frame().reset_index()


In [20]:
df_division=pd.merge(df_division_sales,df_division_trans,on=['transaction_dt','DCart_Type_all','group','division_id'],how="outer")
df_division['AOV']=df_division['item_transaction_amt']/df_division['trans']
df_division=df_division.rename(columns={"customer_id_hashed":"division_shoppers","item_transaction_amt":"division_sales","trans":"division_trans","AOV":"division_AOV"})

df_division=df_division.pivot_table(index=['transaction_dt','DCart_Type_all','group'],columns='division_id',values=['division_shoppers','division_sales','division_trans',"division_AOV"]).reset_index()

new_cols=[]
for col in df_division.columns.tolist():
    if not col[1]:
        new_cols.append(col[0])
    else:
        new_cols.append("division_"+col[1]+"_"+col[0].split("_")[1])
df_division.columns=new_cols

new_cols=['transaction_dt','group','DCart_Type_all']
for i in range(10):
    new_cols.append("division_"+str(i)+"_shoppers")
    new_cols.append("division_"+str(i)+"_sales")
    new_cols.append("division_"+str(i)+"_trans")
    new_cols.append("division_"+str(i)+"_AOV")
new_cols=[x for x in new_cols if x in df_division.columns.tolist()]    
df_division=df_division[new_cols]

In [21]:
pd.options.display.max_columns=99

df_output_overall_dcart=pd.merge(df_total,df_division,on=['transaction_dt','group','DCart_Type_all'],how="outer")

df_output_overall_dcart=pd.merge(df_total_id_overall,df_output_overall_dcart,on=["group",'DCart_Type_all'],how="outer")
df_output_overall_dcart=df_output_overall_dcart.sort_values(["transaction_dt","group",'DCart_Type_all'])


In [27]:
df_output_overall_dcart.head(2)

,group,DCart_Type_all,customer_id_hashed,transaction_dt,total_shoppers,total_sales,total_trans,total_AOV,division_1_shoppers,division_1_sales,division_1_trans,division_1_AOV,division_2_shoppers,division_2_sales,division_2_trans,division_2_AOV,division_3_shoppers,division_3_sales,division_3_trans,division_3_AOV,division_4_shoppers,division_4_sales,division_4_trans,division_4_AOV,division_5_shoppers,division_5_sales,division_5_trans,division_5_AOV,division_6_shoppers,division_6_sales,division_6_trans,division_6_AOV,division_9_shoppers,division_9_sales,division_9_trans,division_9_AOV
0,control,NotDCart_but_AgilOne,39454.0,2020-01-12,348,26392.05,388,68.020747,202,2065.79,209,9.884163,171,2710.55,174,15.577874,67,700.91,71,9.871972,133,4290.08,141,30.426099,82,1553.04,82,18.939512,79,837.14,80,10.464250,52,14234.19,53,268.569623
14,control,OverlapWithDCart,14976.0,2020-01-12,446,33493.35,518,64.658977,260,2352.79,273,8.618278,221,3281.26,235,13.962809,86,1703.25,90,18.925000,194,5195.14,209,24.857129,110,1661.88,115,14.451130,86,782.35,90,8.692778,88,18514.48,92,201.244348


# Campaing_3_Days, Overall DCartOverlap

In [28]:
df_sales_3_campaign_days=df_sales_by_day[df_sales_by_day['transaction_dt'].isin(['2020-01-18','2020-01-19','2020-01-20'])]
df_sales_3_campaign_days=df_sales_3_campaign_days.rename(columns={"transaction_dt":"date_kept_to_dedup"})

df_sales_3_campaign_days['transaction_dt']='total_3_days_Jan_18-20'

df_total_sales=df_sales_3_campaign_days.groupby(['transaction_dt','group','DCart_Type_all'])['customer_id_hashed','item_transaction_amt'].agg(agg_fun).reset_index()
df_total_trans=df_sales_3_campaign_days[['group','DCart_Type_all','location_id','transaction_dt','transaction_id','customer_id_hashed','date_kept_to_dedup']].drop_duplicates()
df_total_trans['trans']=1
df_total_trans=df_total_trans.groupby(['transaction_dt','group','DCart_Type_all'])['trans'].sum().to_frame().reset_index()
df_total=pd.merge(df_total_sales,df_total_trans,on=['transaction_dt','group','DCart_Type_all'],how="outer")
df_total=df_total.rename(columns={"customer_id_hashed":"total_shoppers","item_transaction_amt":"total_sales","trans":"total_trans"})
df_total['total_AOV']=df_total['total_sales']/df_total['total_trans']

df_division_sales=df_sales_3_campaign_days.groupby(['transaction_dt','group','DCart_Type_all','division_id'])['customer_id_hashed','item_transaction_amt'].agg(agg_fun).reset_index()
df_division_trans=df_sales_3_campaign_days[['group','DCart_Type_all','location_id','transaction_dt','transaction_id','customer_id_hashed','division_id','date_kept_to_dedup']].drop_duplicates()
df_division_trans['trans']=1
df_division_trans=df_division_trans.groupby(['transaction_dt','group','DCart_Type_all','division_id'])['trans'].sum().to_frame().reset_index()



In [29]:
# 3 days overall

df_division=pd.merge(df_division_sales,df_division_trans,on=['transaction_dt','group','DCart_Type_all','division_id'],how="outer")
df_division['AOV']=df_division['item_transaction_amt']/df_division['trans']
df_division=df_division.rename(columns={"customer_id_hashed":"division_shoppers","item_transaction_amt":"division_sales","trans":"division_trans","AOV":"division_AOV"})

df_division=df_division.pivot_table(index=['transaction_dt','group','DCart_Type_all'],columns='division_id',values=['division_shoppers','division_sales','division_trans',"division_AOV"]).reset_index()

new_cols=[]
for col in df_division.columns.tolist():
    if not col[1]:
        new_cols.append(col[0])
    else:
        new_cols.append("division_"+col[1]+"_"+col[0].split("_")[1])
df_division.columns=new_cols

new_cols=['transaction_dt','group','DCart_Type_all']
for i in range(10):
    new_cols.append("division_"+str(i)+"_shoppers")
    new_cols.append("division_"+str(i)+"_sales")
    new_cols.append("division_"+str(i)+"_trans")
    new_cols.append("division_"+str(i)+"_AOV")
new_cols=[x for x in new_cols if x in df_division.columns.tolist()]    
df_division=df_division[new_cols]

In [30]:
df_output_2=pd.merge(df_total,df_division,on=['transaction_dt','group','DCart_Type_all'],how="outer")

df_output_2=pd.merge(df_total_id_overall,df_output_2,on=["group",'DCart_Type_all'],how="outer")
df_output_2=df_output_2.sort_values(["transaction_dt","group",'DCart_Type_all'])

df_output_overall_dcart=df_output_overall_dcart.append(df_output_2)

In [31]:
'''
df_division_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Division Names.txt",sep="|")

writer=pd.ExcelWriter("./BL_performance_of_furniture_browser_groups_updated_3_Campaign_days_and_DCartOverlap_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_output.to_excel(writer,"view_no_store_type",index=False)
df_division_name.to_excel(writer,"divison_name",index=False)
writer.save()
'''

'\ndf_division_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Division Names.txt",sep="|")\n\nwriter=pd.ExcelWriter("./BL_performance_of_furniture_browser_groups_updated_3_Campaign_days_and_DCartOverlap_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")\ndf_output.to_excel(writer,"view_no_store_type",index=False)\ndf_division_name.to_excel(writer,"divison_name",index=False)\nwriter.save()\n'

# By_Day, 2020 DCartOverlap

In [32]:
agg_fun={"customer_id_hashed":"nunique","item_transaction_amt":"sum"}
df_total_sales=df_sales_by_day.groupby(['transaction_dt','group','DCart_Type_2020'])['customer_id_hashed','item_transaction_amt'].agg(agg_fun).reset_index()
df_total_trans=df_sales_by_day[['group','DCart_Type_2020','location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
df_total_trans['trans']=1
df_total_trans=df_total_trans.groupby(['transaction_dt','DCart_Type_2020','group'])['trans'].sum().to_frame().reset_index()
df_total=pd.merge(df_total_sales,df_total_trans,on=['transaction_dt','DCart_Type_2020','group'],how="outer")
df_total=df_total.rename(columns={"customer_id_hashed":"total_shoppers","item_transaction_amt":"total_sales","trans":"total_trans"})
df_total['total_AOV']=df_total['total_sales']/df_total['total_trans']

df_division_sales=df_sales_by_day.groupby(['transaction_dt','group','DCart_Type_2020','division_id'])['customer_id_hashed','item_transaction_amt'].agg(agg_fun).reset_index()
df_division_trans=df_sales_by_day[['group','DCart_Type_2020','location_id','transaction_dt','transaction_id','customer_id_hashed','division_id']].drop_duplicates()
df_division_trans['trans']=1
df_division_trans=df_division_trans.groupby(['transaction_dt','DCart_Type_2020','group','division_id'])['trans'].sum().to_frame().reset_index()


In [33]:
df_division=pd.merge(df_division_sales,df_division_trans,on=['transaction_dt','DCart_Type_2020','group','division_id'],how="outer")
df_division['AOV']=df_division['item_transaction_amt']/df_division['trans']
df_division=df_division.rename(columns={"customer_id_hashed":"division_shoppers","item_transaction_amt":"division_sales","trans":"division_trans","AOV":"division_AOV"})

df_division=df_division.pivot_table(index=['transaction_dt','DCart_Type_2020','group'],columns='division_id',values=['division_shoppers','division_sales','division_trans',"division_AOV"]).reset_index()

new_cols=[]
for col in df_division.columns.tolist():
    if not col[1]:
        new_cols.append(col[0])
    else:
        new_cols.append("division_"+col[1]+"_"+col[0].split("_")[1])
df_division.columns=new_cols

new_cols=['transaction_dt','group','DCart_Type_2020']
for i in range(10):
    new_cols.append("division_"+str(i)+"_shoppers")
    new_cols.append("division_"+str(i)+"_sales")
    new_cols.append("division_"+str(i)+"_trans")
    new_cols.append("division_"+str(i)+"_AOV")
new_cols=[x for x in new_cols if x in df_division.columns.tolist()]    
df_division=df_division[new_cols]

In [34]:
pd.options.display.max_columns=99

df_output_2020_dcart=pd.merge(df_total,df_division,on=['transaction_dt','group','DCart_Type_2020'],how="outer")

df_output_2020_dcart=pd.merge(df_total_id_2020,df_output_2020_dcart,on=["group",'DCart_Type_2020'],how="outer")
df_output_2020_dcart=df_output_2020_dcart.sort_values(["transaction_dt","group",'DCart_Type_2020'])


# Campaing_3_Days, 2020 DCartOverlap

In [35]:
df_sales_3_campaign_days=df_sales_by_day[df_sales_by_day['transaction_dt'].isin(['2020-01-18','2020-01-19','2020-01-20'])]
df_sales_3_campaign_days=df_sales_3_campaign_days.rename(columns={"transaction_dt":"date_kept_to_dedup"})

df_sales_3_campaign_days['transaction_dt']='total_3_days_Jan_18-20'

df_total_sales=df_sales_3_campaign_days.groupby(['transaction_dt','group','DCart_Type_2020'])['customer_id_hashed','item_transaction_amt'].agg(agg_fun).reset_index()
df_total_trans=df_sales_3_campaign_days[['group','DCart_Type_2020','location_id','transaction_dt','transaction_id','customer_id_hashed','date_kept_to_dedup']].drop_duplicates()
df_total_trans['trans']=1
df_total_trans=df_total_trans.groupby(['transaction_dt','group','DCart_Type_2020'])['trans'].sum().to_frame().reset_index()
df_total=pd.merge(df_total_sales,df_total_trans,on=['transaction_dt','group','DCart_Type_2020'],how="outer")
df_total=df_total.rename(columns={"customer_id_hashed":"total_shoppers","item_transaction_amt":"total_sales","trans":"total_trans"})
df_total['total_AOV']=df_total['total_sales']/df_total['total_trans']

df_division_sales=df_sales_3_campaign_days.groupby(['transaction_dt','group','DCart_Type_2020','division_id'])['customer_id_hashed','item_transaction_amt'].agg(agg_fun).reset_index()
df_division_trans=df_sales_3_campaign_days[['group','DCart_Type_2020','location_id','transaction_dt','transaction_id','customer_id_hashed','division_id','date_kept_to_dedup']].drop_duplicates()
df_division_trans['trans']=1
df_division_trans=df_division_trans.groupby(['transaction_dt','group','DCart_Type_2020','division_id'])['trans'].sum().to_frame().reset_index()



In [36]:
# 3 days overall

df_division=pd.merge(df_division_sales,df_division_trans,on=['transaction_dt','group','DCart_Type_2020','division_id'],how="outer")
df_division['AOV']=df_division['item_transaction_amt']/df_division['trans']
df_division=df_division.rename(columns={"customer_id_hashed":"division_shoppers","item_transaction_amt":"division_sales","trans":"division_trans","AOV":"division_AOV"})

df_division=df_division.pivot_table(index=['transaction_dt','group','DCart_Type_2020'],columns='division_id',values=['division_shoppers','division_sales','division_trans',"division_AOV"]).reset_index()

new_cols=[]
for col in df_division.columns.tolist():
    if not col[1]:
        new_cols.append(col[0])
    else:
        new_cols.append("division_"+col[1]+"_"+col[0].split("_")[1])
df_division.columns=new_cols

new_cols=['transaction_dt','group','DCart_Type_2020']
for i in range(10):
    new_cols.append("division_"+str(i)+"_shoppers")
    new_cols.append("division_"+str(i)+"_sales")
    new_cols.append("division_"+str(i)+"_trans")
    new_cols.append("division_"+str(i)+"_AOV")
new_cols=[x for x in new_cols if x in df_division.columns.tolist()]
df_division=df_division[new_cols]

In [37]:
df_output_2=pd.merge(df_total,df_division,on=['transaction_dt','group','DCart_Type_2020'],how="outer")

df_output_2=pd.merge(df_total_id_2020,df_output_2,on=["group",'DCart_Type_2020'],how="outer")
df_output_2=df_output_2.sort_values(["transaction_dt","group",'DCart_Type_2020'])

df_output_2020_dcart=df_output_2020_dcart.append(df_output_2)

In [41]:
df_division_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Division Names.txt",sep="|")

writer=pd.ExcelWriter("./BL_performance_of_furniture_browser_groups_updated_3_Campaign_days_and_DCartOverlap_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_output_overall_dcart.to_excel(writer,"view_all_overlap",index=False)
df_output_2020_dcart.to_excel(writer,"view_overlap_2020",index=False)
df_division_name.to_excel(writer,"divison_name",index=False)
df_campaing_dates.to_excel(writer,"DCart_All_Campaigns",index=False)
df_2_weeks_campaign.to_excel(writer,"DCart_2020_Campaigns",index=False)

writer.save()

In [ ]:
# To add
# 1. Q4 control
# 2. Broyhill vs non-broyhill